In [163]:
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Input, Flatten , Concatenate
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.utils import plot_model #모델 시각화 
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, TensorBoard


tf.config.run_functions_eagerly(False)



## 데이터 불러오기
- 앞서 크롤링하여 엑셀에 저장한 파일을 불러온다.

In [164]:
data_df = pd.read_excel('baseball_data.xlsx')
target_df = pd.read_excel('label.xlsx')
try:
    del data_df['Unnamed: 0']
    del target_df['Unnamed: 0']
except:
    pass


## 훈련세트, 검증세트, 테스트세트 분리
- 8:2비율로 훈련세트, 테스트 세트를 분리하고 또 다시 8:2로 훈련세트, 검증세트를 분리한다.

In [165]:
#훈련세트, 검증세트, 테스트세트 분리 
X_full_set, X_test=  train_test_split(data_df, test_size = 0.2, random_state = 7)
y_full_set, y_test = train_test_split(target_df, test_size = 0.2, random_state = 7)
x_train, x_val, y_train, y_val = train_test_split(X_full_set,y_full_set, test_size=0.2, random_state=7)
display(x_train)

,타석,타수,안타,홈런,볼넷,사구,삼진,땅볼,플라이,팀타율,출루율,잔루,상대선발투수ERA,상대선발투수WHIP
1362,42,34,10,2,6,1,9,9,6,0.294118,0.309091,9,9.55,2.19
3844,37,33,7,0,2,2,12,8,6,0.212121,0.234043,12,4.56,1.51
3695,37,34,7,1,3,0,10,9,8,0.205882,0.208333,19,2.45,1.17
3424,37,35,10,1,1,1,10,9,6,0.285714,0.266667,15,6.57,1.91
2934,46,38,11,0,7,1,7,12,8,0.289474,0.306452,30,4.82,1.35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3532,37,36,11,1,0,1,6,6,13,0.305556,0.235294,11,2.72,1.11
84,39,36,10,0,2,0,7,9,10,0.277778,0.235294,15,1.83,1.00
1600,48,44,19,1,4,0,12,7,6,0.431818,0.396552,22,8.10,2.10
1287,36,34,9,2,2,0,8,5,12,0.264706,0.220000,13,4.35,1.43


## 데이터 전처리
- min-max 스케일링(정규화) 와 표준화 둘 중 더 좋은걸로 선택
- 전처리 하지 않는걸로...

In [166]:
#데이터 정규화
#scaler = StandardScaler() 
#mms = MinMaxScaler()

#x_train = scaler.fit_transform(x_train)
#x_val=scaler.fit_transform(x_val)
#X_test=scaler.fit_transform(X_test)
#x_val=scaler.fit_transform(x_val)

In [167]:
#차원축소
#pca = PCA(n_components=7)
#X_train = pca.fit_transform(X_train)
#X_valid = pca.fit_transform(X_valid)


## 모델 만들기
- 모델(점수예측)은 회귀모델이기 때문에 출력층에 활성화함수를 지정하지 않았다.

In [168]:
#모델 만들기
input = Input(shape=(14,),name='input')
hidden1 = Dense(400,activation='relu',name='hidden1')(input)
hidden2 = Dense(200,activation='relu',name='hidden2')(hidden1)
hidden3= Dense(50,activation='relu',name='hidden3')(hidden2)
hidden4 = Dense(30,activation='relu',name='hidden_4')(hidden3)
output = Dense(1,name='output')(hidden4)


model = Model(inputs=input,outputs=output)



## 모델컴파일
- 회귀문제 : MSE(loss) , MAE(eval)
- 이진분류문제 : binary cross entropy(loss) , accuracy(eval)

In [169]:
model.compile(loss='mse',
              optimizer=Adam(learning_rate=1e-2),metrics=['mae'])


# 텐서보드

In [170]:
TensorBoard(log_dir='.logs', histogram_freq=0, write_graph=True, write_images=True)

In [171]:
log_dir='./logs'
tensor_board_cb = [TensorBoard(log_dir=log_dir, histogram_freq=1,write_graph=True,write_images=True)]

## 모델학습 및 저장

In [172]:
#최상의 모델을 저장 
cp= ModelCheckpoint('score_predict_best_model.h5', save_best_only=True)
early_Stopping_cb = EarlyStopping(patience=10, monitor='val_loss',restore_best_weights=True)
history = model.fit(x_train,
                    y_train,
                    epochs=100, batch_size=30, validation_data=(x_val,y_val), callbacks=[cp,early_Stopping_cb,tensor_board_cb])


Epoch 1/100
83/83 [==============================] - 2s 9ms/step - loss: 12.7819 - mae: 2.2486 - val_loss: 3.8592 - val_mae: 1.4991
Epoch 2/100
83/83 [==============================] - 0s 5ms/step - loss: 3.0777 - mae: 1.3662 - val_loss: 3.8764 - val_mae: 1.4881
Epoch 3/100
83/83 [==============================] - 0s 6ms/step - loss: 2.7988 - mae: 1.2954 - val_loss: 3.5966 - val_mae: 1.4425
Epoch 4/100
83/83 [==============================] - 0s 5ms/step - loss: 2.6696 - mae: 1.2754 - val_loss: 2.5110 - val_mae: 1.2156
Epoch 5/100
83/83 [==============================] - 0s 5ms/step - loss: 3.1662 - mae: 1.3818 - val_loss: 3.0703 - val_mae: 1.3260
Epoch 6/100
83/83 [==============================] - 0s 4ms/step - loss: 2.7061 - mae: 1.2856 - val_loss: 2.8457 - val_mae: 1.2769
Epoch 7/100
83/83 [==============================] - 0s 4ms/step - loss: 2.6577 - mae: 1.2619 - val_loss: 2.5338 - val_mae: 1.2220
Epoch 8/100
83/83 [==============================] - 0s 5ms/step - loss: 2.7666 - 

In [173]:
%load_ext tensorboard
%tensorboard --logdir {log_dir}

## 모델 평가

In [174]:
model.evaluate(X_test,y_test)


25/25 [==============================] - 0s 1ms/step - loss: 2.3788 - mae: 1.1869


[2.3788340091705322, 1.186936378479004]

## 예측
- 학습하지 않은 22년 10월 8일 3경기의 데이터를 예측하고자 한다.
- 단 이때 22년 10월 8일의 경기가 진행되지 않음을 가정하고 예측을 진행한다. 스코어예측이나 점수예측은 주로 경기 전에 진행되는
  특성을 따른 것.
- 때문에 모델에 input으로 들어가는 데이터는 각 구단의 직전 3경기의 평균값을 입력하고 실제 22년 10월 8일의 데이터와 비교한다.
- 선발투수는 경기전 확정된 선발투수의 데이터로 입력값을 넣는다(직전 3경기 평균값X 당일 데이터O)
|22-10-8|A구단 점수|B구단 점수|
|------|----------|----------|
|SSG(A) : 삼성(B)|1|6|
|키움 : 두산|5|1|
|한화 : NC|5|6|


In [175]:
#예측1 SSG:삼성
SSG =[36.3,33.6,6.6,1,2.3,0,9.6,10,7.6,0.196,0.153,18,2.49,1.16]
samsung = [41.6,38.6,12.6,0.3,2.3,0,8,8.6,9.3,0.325,0.252,19,6.00,1.52]
dusan = [34.6,32.6,9.3,1,1.3,0,6.3,8,9,0.282,0.213,11.6,3.60,1.33]
kiwum = [50,44,12,1,5.3,0.6,8.6,11.6,11.6,0.268,0.213,21.6,2.11,0.95]
hanwha = [42.6,34.6,9,0,5.6,1,6.3,10.6,9,0.258,0.201,24.6,4.75,1.42]
NC = [34.3,30.6,6.6,0.3,3,2,8.6,9,7,0.216,0.198,11.6,10.64,1.91]

score_SSG_samsung =model.predict([SSG,samsung])
score_dusan_kiwum =model.predict([dusan,kiwum])
score_hanwha_NC =model.predict([hanwha,NC])

print(f"SSG 예측점수:{score_SSG_samsung[0]}")
print(f"삼성 예측점수:{score_SSG_samsung[1]}")
print(f"두산 예측점수:{score_dusan_kiwum[0]}")
print(f"키움 예측점수:{score_dusan_kiwum[1]}")
print(f"한화 예측점수:{score_hanwha_NC[0]}")
print(f"NC 예측점수:{score_hanwha_NC[1]}")


1/1 [==============================] - 0s 79ms/step
SSG 예측점수:[1.7606862]
삼성 예측점수:[5.3365083]
두산 예측점수:[3.9001179]
키움 예측점수:[6.48838]
한화 예측점수:[4.058975]
NC 예측점수:[3.8667593]
